In [ ]:
# Copyright 2023 Intel Corporation
# SPDX-License-Identifier: MIT

<h1 style="font-size: 22px; line-height: 100%; text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142); border-radius: 10px;">
External Sub-network Search with BootstrapNAS and DyNAS-T
</h1>

This notebook demonstrates how to use an external search solution to find high-performing subnetworks in NNCF's [BootstrapNAS](https://github.com/openvinotoolkit/nncf/tree/develop/examples/experimental/torch/classification). We will use an existing super-network generated from ResNet-50.  

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Imports and Settings
</h3>

In [ ]:
import sys
import os
sys.path.append('..')
sys.path.append('path-to-nncf')

from imports_bnas import * # Import NNCF, PyTorch and other required packages.
import os
from examples.common.sample_config import SampleConfig
from nncf.common.initialization.batchnorm_adaptation import BatchnormAdaptationAlgorithm
from nncf.experimental.torch.nas.bootstrapNAS.search.supernet import TrainedSuperNet
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.multi_elasticity_handler import SubnetConfig
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.elasticity_dim import ElasticityDim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

MODEL_DIR = Path("../../../models/supernets/cifar10/resnet50")
OUTPUT_DIR = Path("output")
DATA_DIR = Path("data")
BASE_MODEL_NAME = "resnet-50"

fp32_pth_path, model_onnx_path, supernet_onnx_path, subnet_onnx_path = create_folders_demo(BASE_MODEL_NAME + '_external_search')


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Loading an existing super-network
</h3>

In [ ]:
model = resnet50_cifar10()
config = SampleConfig.from_json(os.path.join(MODEL_DIR, 'config.json'))
config.device = device
print(MODEL_DIR)
supernet = TrainedSuperNet.from_checkpoint(model, config, os.path.join(MODEL_DIR, 'elasticity.pth'), os.path.join(MODEL_DIR, 'supernet_weights.pth'))

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Explore the Search Space
</h3>



In [ ]:
supernet.get_search_space()

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Data Loaders for validation and batchnorm adaptation
</h3>

Note: Number of samples used for Batch Norm Adaptation might greatly influence the top1 evaluation for each subnetwork.
`6000` is a good value for ResNet-50 CIFAR10, but it is advised to perform an ablation study for each new architecture to identify the best number of samples.

In [ ]:
DATASET_DIR = DATA_DIR / "cifar10"

batch_size_val = 1000
batch_size = 64
train_loader, val_loader = create_cifar10_dataloader(DATASET_DIR, batch_size, batch_size_val, device)

bn_adapt_args = BNAdaptationInitArgs(data_loader=wrap_dataloader_for_init(train_loader), device=config.device)

bn_adapt_algo_kwargs = {
    'data_loader': train_loader,
    'num_bn_adaptation_samples': 6000,
    'device': 'cuda',
}
bn_adaptation = BatchnormAdaptationAlgorithm(**bn_adapt_algo_kwargs)

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Sample sub-networks to implement your own algorithm
</h3>


In [ ]:
# Implement your function to validate 

def acc_top1(model):
    from bootstrapnas_utils import validate

    bn_adaptation.run(model)

    model.eval()
    model.to(device)
    model_top1_acc, _, _ = validate(model, val_loader)
    return model_top1_acc.detach().cpu().numpy()

supernet.activate_maximal_subnet()
supernet.eval_active_subnet(acc_top1)

In [ ]:
subnet_config = SubnetConfig()
subnet_config[ElasticityDim.DEPTH] = [1]

supernet.eval_subnet(subnet_config, acc_top1)

In [ ]:
supernet.activate_config(supernet.get_search_space())
supernet.get_macs_for_active_config()

In [ ]:
supernet.activate_config(supernet.get_search_space())
supernet.get_macs_for_active_config()

In [ ]:
subnet_config[ElasticityDim.DEPTH] = [0]
subnet_config[ElasticityDim.WIDTH] = {2: 512}

supernet.activate_config(subnet_config)
supernet.get_macs_for_active_config()

In [ ]:
supernet.get_active_config()

In [ ]:
supernet.eval_subnet(subnet_config, acc_top1)

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Search using DyNAS-T
</h3>

![DyNAS-T Logo](https://github.com/IntelLabs/DyNAS-T/blob/main/docs/images/dynast_logo.png?raw=true)

DyNAS-T (Dynamic Neural Architecture Search Toolkit) is a super-network neural architecture search NAS optimization package designed for efficiently discovering optimal deep neural network (DNN) architectures for a variety of performance objectives such as accuracy, latency, multiply-and-accumulates, and model size.

To install DyNAS-T you can either install it from PyPI or from source:

1. PyPI: In your active Python environment install DyNAS-T with:
    ```bash
    pip install dynast
    ```
1. Source: Clone DyNAS-T repository with:
    ```bash
    git clone https://github.com/IntelLabs/DyNAS-T.git
    ``` 
    and from the cloned repository run
    ```bash
    pip install -e .
    ```

DyNAS-T is compatible with supernetworks created with the NNCF BootstrapNAS. The search on NNCF BootstrapNAS supernetworks can be performed in just few steps, as shown below.

In [ ]:
from dynast.dynast_manager import DyNAS

dynast_config = {
    'results_path': 'bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv',
    'num_evals': 250,
    'population': 50,
    'seed': 42,
    'supernet': 'bootstrapnas_image_classification',  # this parameter works in a tangent with `bootstrapnas_supernetwork` and should be set to `bootstrapnas_image_classification` for all CNN BootstrapNAS supernetworks;
    'batch_size': config.batch_size,
    'bootstrapnas_supernetwork': supernet,  # pass the instance of nncf.experimental.torch.nas.bootstrapNAS.search.supernet.TrainedSuperNet to the search algorithm;
    'device': config.device,
    'verbose': False,
    'optimization_metrics': ['accuracy_top1', 'macs'],  # define optimization metrics that will guide the search progression;
    'measurements': ['accuracy_top1', 'macs'],  # define measurements that will be collected during the search; This is a superset of optimization metrics;
    'metric_eval_fns': {
        'accuracy_top1': acc_top1,  # register custom evaluation function;
    }
}

dynas = DyNAS(**dynast_config)

In [ ]:
# `results` will contain a list of <population> best configurations found during the search;
results = dynas.search()

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Visualization of DyNAS-T search on the BootstrapNAS supernetwork
</h3>

<center><img src="dynast_bootstrapnas_resnet50_cifar10_example.png" alt="DyNAS-T Search Results" width="1000"/></center>

In [ ]:
# `results` will contain a list of <population> best configurations found during the search;
results = dynas.search()

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Visualization of DyNAS-T search on the BootstrapNAS supernetwork
</h3>

<center><img src="dynast_bootstrapnas_resnet50_cifar10_example.png" alt="DyNAS-T Search Results" width="1000"/></center>